### 声明

* 该代码源自：https://github.com/mhsamavatian/DAP/blob/master/1-GenerateFeatureVector

* 由于我们下载的数据集有所更新、我们希望提取的特征也有所不同，所以我们对原始代码进行了一定的改动

### 代码功能

* 载入交通事件数据: https://smoosavi.org/datasets/lstw

* 载入气象和光照数据：https://github.com/mhsamavatian/DAP/tree/master/data

* 将研究区域栅格化为5km*5km的单元格，并给每个小单元格分配一个geohash

* 把研究的时间段切分成以time_interval分钟为间隔的时间步

* 对于每个geohash和每个时间步，初步生成其特征向量（包含时间、交通事件、气候）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pygeohash
!pip install haversine

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import pytz
import pygeohash as gh
from haversine import haversine
import time
import json
import math
from sklearn.preprocessing import OneHotEncoder

grid_width = 5 # 栅格化粒度（每个小单元格的边长）
time_interval = 15 # 时间间隔

In [ ]:
cities = {'Austin': [30.079327, 30.596764,-97.968881,-97.504838]}

time_zones = {'Austin':'US/Central'}

# 研究时间间隔
start = datetime(2018, 6, 1)
finish   = datetime(2018, 9, 2)

begin = datetime.strptime('2018-06-01 00:00:00', '%Y-%m-%d %H:%M:%S')
end   = datetime.strptime('2018-08-31 23:59:59', '%Y-%m-%d %H:%M:%S')

### 载入交通事件数据

In [ ]:
! tar -zxvf /content/drive/MyDrive/Capstone/data/TrafficEvents_Aug16_Dec20_Publish.tar.gz

TrafficEvents_Aug16_Dec20_Publish.csv


In [ ]:
# 由于新数据集的数据量太大，我们需要分块读取

chunksize=1000000

traffic_event = pd.DataFrame()

for temp_event in pd.read_csv('TrafficEvents_Aug16_Dec20_Publish.csv',chunksize=chunksize):
  temp_event['StartTime(UTC)'] = temp_event['StartTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')
  temp_event['EndTime(UTC)'] = temp_event['EndTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')
  if (min(temp_event['EndTime(UTC)']) <= end and max(temp_event['StartTime(UTC)']) >= start):
    traffic_event = pd.concat([traffic_event,temp_event], ignore_index=True)

In [ ]:
# weather_event = pd.read_csv(u'drive/MyDrive/Capstone/data/WeatherEvents_Aug16_Dec20_Publish.csv')
# weather_event.head()

# weather_event['StartTime(UTC)'] = weather_event['StartTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')
# weather_event['EndTime(UTC)'] = weather_event['EndTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')

In [ ]:
traffic_event['StartTime(UTC)'] = traffic_event['StartTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')
traffic_event['EndTime(UTC)'] = traffic_event['EndTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')
# weather_event['StartTime(UTC)'] = weather_event['StartTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')
# weather_event['EndTime(UTC)'] = weather_event['EndTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')

In [ ]:
for c in cities:
    crds = cities[c]
    subset_all = traffic_event[(traffic_event['StartTime(UTC)'] >= start) & (traffic_event['StartTime(UTC)'] < end) & 
                    (traffic_event['LocationLat']>crds[0]) & (traffic_event['LocationLat']<crds[1]) & (traffic_event['LocationLng']>crds[2]) & 
                    (traffic_event['LocationLng']<crds[3])]
    
    subset_accidents = traffic_event[(traffic_event['Type']=='Accident') & (traffic_event['StartTime(UTC)'] >= start) & (traffic_event['StartTime(UTC)'] < finish) 
                          & (traffic_event['LocationLat']>crds[0]) & (traffic_event['LocationLat']<crds[1]) & (traffic_event['LocationLng']>crds[2]) 
                          & (traffic_event['LocationLng']<crds[3])]
    
    print('For {} we have {} incidents, with {} accidents! ratio {:.2f}'.format(c, len(subset_all), len(subset_accidents), len(subset_accidents)*1.0/len(subset_all)))
    
    subset_all.to_csv(u'drive/MyDrive/Capstone/data/traffic_event_data.csv', index=False)

For Austin we have 20435 incidents, with 4413 accidents! ratio 0.22


In [ ]:
# code_number = {}

# for c in cities:
#     with open(path + 'MQ_{}_20180601_20180609.csv'.format(c), 'r') as file:
#         header = False
#         for line in file:
#             if not header:
#                 header = True
#                 continue
#             parts = line.replace('\r', '').replace('\n', '').split(',')
#             temp =  code_conversion[code_conversion.Code==float(parts[3])].C.values
#             if (len(temp)>0):
#               if int(temp[0]) in code_number:
#                 code_number[int(temp[0])] += 1
#               else:
#                 code_number[int(temp[0])] = 1

code_conversion = pd.read_csv(u'drive/MyDrive/Capstone/data/event_code.csv')

name_conversion = {}

for c in cities:
    with open(u'drive/MyDrive/Capstone/data/traffic_event_data.csv', 'r') as file:
        header = False
        for line in file:
            if not header:
                header = True
                continue
            parts = line.replace('\r', '').replace('\n', '').split(',')
            code = parts[3]
            temp = code_conversion[code_conversion.Code==int(code)].C.values
            if (len(temp)>0):
              C = temp[0]
              if (C == 1 or C == 20):
                name_conversion[code]='Congestion'
              elif (C == 3):
                name_conversion[code]='Accident'
              elif (C == 4):
                name_conversion[code]='Incident'
              elif (C == 5 or C == 7 or C == 8 or C == 9):
                name_conversion[code]='Restriction'
              elif (C == 11 or C == 12 or C == 13):
                name_conversion[code]='Obstruction'
              elif (C == 18):
                name_conversion[code]='Activity'
              elif (C == 25):
                name_conversion[code]='Equipment'
              else:
                name_conversion[code]='Other'

### 将交通事件数据匹配至每个单元格内

In [ ]:
zone_to_be = {}

for z in ['US/Central']:
    t_begin = begin.replace(tzinfo=pytz.timezone(z))
    t_end   = end.replace(tzinfo=pytz.timezone(z))
    zone_to_be[z] = [t_begin, t_end]

In [ ]:
def return_interval_index(time_stamp, start, end):
    if time_stamp < start or time_stamp > end: 
        return -1
    index = int(((time_stamp-start).days*24*60 + (time_stamp-start).seconds/60)/time_interval)
    return index

diff = math.ceil(((end-begin).days*24*60 + (end-begin).seconds/60)/time_interval)

city_to_geohashes = {}
for c in cities: city_to_geohashes[c] = {}

start_timestamp = time.time()

geocode_to_airport = {}
aiport_to_timezone = {}

for c in cities:
    z = time_zones[c]
    
    with open(u'drive/MyDrive/Capstone/data/traffic_event_data.csv', 'r') as file:
        header = False
        for line in file:
            if not header:
                header = True
                continue
            parts = line.replace('\r', '').replace('\n', '').split(',')
            
            ds = datetime.strptime(parts[5].replace('T',' '), '%Y-%m-%d %H:%M:%S')
            ds = ds.replace(tzinfo=pytz.utc)
            ds = ds.astimezone(pytz.timezone(z))
            s_interval = return_interval_index(ds, zone_to_be[z][0], zone_to_be[z][1])
            if s_interval==-1: continue
                
            de = datetime.strptime(parts[6].replace('T',' '), '%Y-%m-%d %H:%M:%S')
            de = de.replace(tzinfo=pytz.utc)
            de = de.astimezone(pytz.timezone(z))
            e_interval = return_interval_index(de, zone_to_be[z][0], zone_to_be[z][1])
            if e_interval == -1: e_interval = diff-1    
            
            start_gh = gh.encode(float(parts[8]), float(parts[9]), precision=grid_width) #事件所在的geohash
            
            intervals = []
            if start_gh not in city_to_geohashes[c]:
                for i in range(diff):
                    intervals.append({'Congestion':0, 'Accident':0, 'Incident':0, 'Restriction':0,
                                      'Obstruction':0, 'Activity':0, 'Equipment':0, 'Other':0})
            else:
                intervals = city_to_geohashes[c][start_gh]
            
            if parts[3] in name_conversion:
                tp = name_conversion[parts[3]]
                
            for i in range(s_interval, e_interval+1):                
                v = intervals[i]
                if tp in v: v[tp] = v[tp] + 1
                else: v['Other'] = v['Other'] + 1
                intervals[i] = v
                
                if tp == 'Accident': break # unlike other types of traffic events
                
            city_to_geohashes[c][start_gh] = intervals #更新事件

            ap = parts[11]
            if len(ap) > 3:
                if start_gh not in geocode_to_airport:
                    geocode_to_airport[start_gh] = set([ap])
                else:
                    st = geocode_to_airport[start_gh]
                    st.add(ap)
                    geocode_to_airport[start_gh] = st
                aiport_to_timezone[ap] = z
  
    
    print ('Done with {} in {:.1f} sec! there are {} geohashes with data!'.format(c, 
                                time.time()-start_timestamp, len(city_to_geohashes[c])))
    start_timestamp = time.time()   

### 载入天气数据

In [ ]:
class weather:
    date = ''
    temp = 0.0
    windchill = 0.0
    humid = 0.0
    pressure= 0.0
    visib = 0.0
    windspeed = 0.0
    winddir = ''
    precipitation = 0.0
    events = ''
    condition = ''
    
    def __init__(self, date, temp, windchill, humid, pressure, visib, windspeed, winddir, 
                 precipitation, events, condition, zone):
        self.date = datetime.strptime(date, '%Y-%m-%d %I:%M:%S %p')
        self.date = self.date.replace(tzinfo=pytz.timezone(zone))
        self.temp = float(temp)
        self.windchill = float(windchill)
        self.humid = float(humid)
        self.pressure = float(pressure)
        self.visib = float(visib)
        self.windspeed = float(windspeed)
        self.winddir = winddir
        self.precipitation = float(precipitation)
        self.events = events
        self.condition = condition

In [ ]:
# load and sort relevant weather data
airports_to_observations = {}
for g in geocode_to_airport:
    aps = geocode_to_airport[g]
    for a in aps:
        if a not in airports_to_observations:
            airports_to_observations[a] = []

print ('{} airports to collect data for!'.format(len(airports_to_observations)))
            
w_path = u'drive/MyDrive/Capstone/data/weather_data/' # this directory contains weather observation records for each airport
airport_to_data = {}
for ap in airports_to_observations:
    data = []
    z = aiport_to_timezone[ap]
    print('Airport {}'.format(ap))
    header = ''
    try:
        with open(w_path + ap + '.csv', 'r') as file:
            for line in file:
                if 'Airport' in line: 
                    header = line.replace('\r','').replace('\n','').replace(',Hour','')
                    continue
                parts = line.replace('\r', '').replace('\n', '').split(',')
                try:
                    w = weather(parts[1] + ' ' + parts[2].split(' ')[0] + ':00 ' + parts[2].split(' ')[1], parts[3], parts[4], 
                              parts[5], parts[6], parts[7], parts[8], parts[9], parts[10], parts[11], parts[12], z)   
                    data.append(w)
                except:
                    continue

            data.sort(key=lambda x:x.date)
            airport_to_data[ap] = data
    except:
        print('Error')
        #把有关这个机场的索引都删除
        del_geocodes = []
        for g in geocode_to_airport:
          aps = geocode_to_airport[g]
          if ap in aps:
            aps.remove(ap)
          geocode_to_airport[g] = aps
          if (len(aps)==0):
            del_geocodes.append(g)
        
        for key in del_geocodes:
          del geocode_to_airport[key]
        
        continue
    
print ('\nData for {} airport stations is loaded!'.format(len(airport_to_data)))

In [ ]:
for c in city_to_geohashes:
    for g in city_to_geohashes[c]:
        if g not in geocode_to_airport:
            print('Found')
            gc = gh.decode_exactly(g)[0:2]
            min_dist = 1000000000
            close_g = ''
            for _g in geocode_to_airport:
                _gc = gh.decode_exactly(_g)[0:2]
                dst = haversine(gc, _gc, 'km')
                if dst < min_dist:
                    min_dist = dst
                    close_g = _g
#             print g, close_g, min_dist
            geocode_to_airport[g] = geocode_to_airport[close_g]

In [ ]:
city_to_geohashes_to_weather = {}

for c in city_to_geohashes: #不同城市
    start = time.time()
    geo2weather = {}
    for g in city_to_geohashes[c]: #不同栅格
        w_data = []
        for i in range(len(city_to_geohashes[c][g])):
            w_data.append({'Temperature':[], 'Humidity':[], 'Pressure':[], 'Visibility':[], 'WindSpeed':[], 
                          'Precipitation':[], 'Condition':set(), 'Event':set()})
        # populate weather data
        aps = geocode_to_airport[g]
        for a in aps: #不同机场
            z = aiport_to_timezone[a]
            a_w_data = airport_to_data[a] #天气数据
            prev = 0
            for a_w_d in a_w_data: #不同时刻的天气
                idx = return_interval_index(a_w_d.date, zone_to_be[z][0], zone_to_be[z][1]) #该时刻对应的tid
                if idx >-1:
                    for i in range(prev, min(idx+1, len(w_data))):
                        _w = w_data[i]
                        
                        _tmp = _w['Temperature']
                        if a_w_d.temp > -1000:
                            _tmp.append(a_w_d.temp)
                            _w['Temperature'] = _tmp
                        
                        _hmd = _w['Humidity']
                        if a_w_d.humid > -1000:
                            _hmd.append(a_w_d.humid)
                            _w['Humidity'] = _hmd
                        
                        _prs = _w['Pressure']
                        if a_w_d.pressure > -1000:
                            _prs.append(a_w_d.pressure)
                            _w['Pressure'] = _prs
                        
                        _vis = _w['Visibility']
                        if a_w_d.visib > -1000:
                            _vis.append(a_w_d.visib)
                            _w['Visibility'] = _vis
                            
                        _wspd = _w['WindSpeed']
                        if a_w_d.windspeed > -1000:
                            _wspd.append(a_w_d.windspeed)
                            _w['WindSpeed'] = _wspd
                            
                        _precip = _w['Precipitation']
                        if a_w_d.precipitation > -1000:
                            _precip.append(a_w_d.precipitation)
                            _w['Precipitation'] = _precip
                            
                        _cond = _w['Condition']
                        _cond.add(a_w_d.condition)
                        _w['Condition'] = _cond
                        
                        _evnt = _w['Event']
                        _evnt.add(a_w_d.events)
                        _w['Event'] = _evnt
                        
                        w_data[i] = _w
                        
                    prev = idx+1
                                                
        geo2weather[g] = w_data
    city_to_geohashes_to_weather[c] = geo2weather
    print('Done with {} in {:.1f} sec!'.format(c, time.time()-start))

### 载入光照数据

In [ ]:
class dayLight:
    sunrise = []
    sunset = []
    def __init__(self, sunrise, sunset):
        self.sunrise = sunrise
        self.sunset = sunset
        
def return_time(x):
    try:
        h = int(x.split(':')[0])
        m = int(x.split(':')[1].split(' ')[0])
        if 'pm' in x and h < 12: h = h + 12
        return [h,m]
    except: return [0,0]

    
def returnDayLight(city, state, dt):
    sc = city + '-' + state
    days = city_days_time[sc]
    d = str(dt.year) + '-' + str(dt.month) + '-' + str(dt.day)
    if d in days:
        r = days[d]
        if ((dt.hour>r.sunrise[0] and dt.hour<r.sunset[0]) or
            (dt.hour>=r.sunrise[0] and dt.minute>=r.sunrise[1] and dt.hour<r.sunset[0]) or
            (dt.hour>r.sunrise[0] and dt.hour<=r.sunset[0] and dt.minute<r.sunset[1]) or 
            (dt.hour>=r.sunrise[0] and dt.minute>=r.sunrise[1] and dt.hour<=r.sunset[0] and dt.minute<r.sunset[1])):
            return '1'
        else: return '0'

In [ ]:
city_days_time = {}

days = {}
city = ''
with open(u'drive/My Drive/Capstone/data/daylight_data.csv', 'r') as file: # you find daylight data for the selected 6 cities in this file
    for ln in file.readlines():
        parts = ln.replace('\r','').replace('\n','').split(',')

        if parts[0] != city:
            if len(city) > 0: 
                if city in city_days_time:
                    _days = city_days_time[city]
                    for _d in _days: days[_d] = _days[_d]
                city_days_time[city] = days

            city = parts[0]
            days = {}

        sunrise = return_time(parts[2])
        sunset  = return_time(parts[3])
        dl = dayLight(sunrise, sunset)
        days[parts[1]] = dl

if city in city_days_time:
    _days = city_days_time[city]
    for _d in _days: days[_d] = _days[_d]
city_days_time[city] = days


print('Successfully loaded daylight data for {} cities!'.format(len(city_days_time)))

In [ ]:
# pre-load daylight mapping for different cities
city_to_index_to_daylight = {}
states = {'Austin':'TX'}
for c in cities:
    d_begin = begin.replace(tzinfo=pytz.timezone(time_zones[c]))
    d_end   = end.replace(tzinfo=pytz.timezone(time_zones[c]))
    index_to_daylight = {}
    index = 0
    while(d_begin < d_end):
        dl = returnDayLight(c, states[c], d_begin)
        index_to_daylight[index] = dl
        index += 1
        d_begin += timedelta(seconds=time_interval*60)
    city_to_index_to_daylight[c] = index_to_daylight

### 数据整合

In [ ]:
# map each time-step to hour of day and day of the week; this should be consistent across different time-zones!
timestep_to_dow_hod = {}
d_begin = begin.replace(tzinfo=pytz.utc)
d_end   = end.replace(tzinfo=pytz.utc)
index = 0

while (d_begin < d_end):
    dow = d_begin.weekday()
    hod = d_begin.hour
    timestep_to_dow_hod[index] = [dow, hod]
    d_begin += timedelta(seconds=time_interval*60)    
    index += 1

In [ ]:
traffic_tags =  ['Accident', 'Congestion', 'Incident', 'Restriction', 'Obstruction', 'Activity', 'Equipment', 'Other']
weather_tags = ['Condition', 'Event', 'Humidity', 'Precipitation', 'Pressure', 'Temperature', 'Visibility', 'WindSpeed']
poi_tags = []
start = time.time()
condition_tags = set()

for c in city_to_geohashes:
    # creating vector for each reion (geohash) during a time_interval minutes time interval. Such vector contains time, traffic, and weather attributes. 
    writer = open(u'temp_data_{}_{}.csv'.format(str(grid_width),str(time_interval)), 'w')
    writer.write('Geohash,TimeStep,DOW,HOD,DayLight,T-Accident,T-Congestion,T-Incident,T-Restriction,'\
        'T-Obstruction,T-Activity,T-Equipment,T-Other,W-Humidity,W-Precipitation,W-Pressure,'\
        'W-Temperature,W-Visibility,W-WindSpeed,W-Rain,W-Snow,W-Fog,W-Hail\n')
    
    traffic = city_to_geohashes[c]
    weather = city_to_geohashes_to_weather[c]        
    for g in traffic:
        vectors = []
        for i in range(len(traffic[g])):
            v = []
            for t in traffic_tags: v.append(traffic[g][i][t])
            v_w = [0,0,0,0] # for rain, snow, fog, and hail
            for w in weather_tags:
                if w=='Condition' or w=='Event':      
                    _tgs = weather[g][i][w]
                    for _tg in _tgs: 
                        if 'rain' in _tg.lower() or 'drizzle' in _tg.lower() or 'thunderstorm' in _tg.lower(): v_w[0] = 1
                        elif 'snow' in _tg.lower(): v_w[1] = 1
                        elif 'fog' in _tg.lower() or 'haze' in _tg.lower() or 'mist' in _tg.lower() or 'smoke' in _tg.lower(): v_w[2] = 1
                        elif 'hail' in _tg.lower() or 'ice pellets' in _tg.lower(): v_w[3] = 1                            
                elif len(weather[g][i][w]) == 0: v.append(0)
                else: v.append(np.mean(weather[g][i][w]))
            for _v_w in v_w: v.append(_v_w)
            vectors.append(v)
        
        for i in range(len(vectors)):
            v = vectors[i]
            v = [str(v[j]) for j in range(len(v))]
            v = ','.join(v)
            writer.write(g + ',' + str(i) + ',' + str(timestep_to_dow_hod[i][0]) + ',' + str(timestep_to_dow_hod[i][1]) 
                         + ',' + city_to_index_to_daylight[c][i] + ',' + v + '\n')
            
    writer.close()
    print ('Done with {} in {:.1f} sec! #vectors {}!'.format(c, time.time()-start, len(traffic)*len(vectors)))
    start = time.time()

In [ ]:
import pickle

geohash_map = pd.read_csv(u'drive/MyDrive/Capstone/data/poi_data.csv')
geo_dict = dict(zip(geohash_map.Geohash.unique(), range(len(geohash_map.Geohash.unique()))))

In [ ]:
df = pd.read_csv(u'temp_data_{}_{}.csv'.format(str(grid_width),str(time_interval)))

print ("zero accident =",float(df[df['T-Accident']==0].shape[0])/df.shape[0])

def fun_hash(geohash):
    return geo_dict[geohash]

df['geohash_code'] = df.apply(lambda row: fun_hash(row['Geohash']), axis=1) 

df = df [[u'TimeStep', u'T-Accident',u'Geohash',u'geohash_code', u'HOD', u'DOW', u'DayLight',
    u'T-Congestion', u'T-Incident', u'T-Restriction', u'T-Obstruction',
    u'T-Activity', u'T-Equipment', u'T-Other', u'W-Humidity',
    u'W-Precipitation', u'W-Pressure', u'W-Temperature', u'W-Visibility',
    u'W-WindSpeed', u'W-Rain', u'W-Snow', u'W-Fog', u'W-Hail']]

def week_day(DOW):
    if DOW < 5:
        return 1
    else:
        return 0
def shift(group):
    df_list=[]
    for idx,df in group:
        df['predicted_accident'] = df['T-Accident'].shift(-1)
        df.drop(df.tail(1).index,inplace=True)
        df_list.append(df)
    return pd.concat(df_list)

def time_category(HOD):
    if HOD >=6 and HOD <10:
        return 0
    if HOD >= 10 and HOD<14:
        return 1
    if HOD >=14 and HOD< 18:
        return 2;
    if HOD >=18 and HOD< 22:
        return 3
    else:
        return 4; 
        
def make_binary(d):
    if d > 0:
        return 1
    else:
        return 0

def onhot_enoceder(df):
     myEncoder = OneHotEncoder(sparse=False)
     myEncoder.fit(df['HOD_cat'].values.reshape(-1, 1))

     onehot_encode = pd.concat([df.reset_index().drop('HOD_cat',1),
                 pd.DataFrame(myEncoder.transform(df['HOD_cat'].values.reshape(-1, 1)),
                              columns=['HOD_en0','HOD_en1','HOD_en2','HOD_en3','HOD_en4'])], axis=1).reindex()

     return onehot_encode.drop('index',1)

df['DOW_cat'] = df.apply(lambda row: week_day(row['DOW']), axis=1)   
df['HOD_cat'] = df.apply(lambda row: time_category(row['HOD']), axis=1) 
df['T-Accident'] = df.apply(lambda row: make_binary(row['T-Accident']), axis=1) 
group = df.groupby('Geohash')
df = shift(group)
df = onhot_enoceder(df)

df = df [[u'TimeStep', u'predicted_accident',u'Geohash',u'geohash_code', u'DOW_cat', u'DayLight',
    u'HOD_en0', u'HOD_en1', u'HOD_en2', u'HOD_en3', u'HOD_en4', u'T-Accident',
    u'T-Congestion', u'T-Incident', u'T-Restriction', u'T-Obstruction',
    u'T-Activity', u'T-Equipment', u'T-Other', u'W-Humidity',
    u'W-Precipitation', u'W-Pressure', u'W-Temperature', u'W-Visibility',
    u'W-WindSpeed', u'W-Rain', u'W-Snow', u'W-Fog', u'W-Hail']]

display (df.head())

df.to_csv(u'drive/MyDrive/Capstone/data/integrated_data_{}_{}.csv'.format(str(grid_width),str(time_interval)))